# Training

> 

In [1]:
#| default_exp training

In [2]:
#| hide
from nbdev.showdoc import *

## Imports and loading datsets

In [12]:
#| export
import torch
import numpy as np
import pandas as pd
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import LayoutLMv3ForTokenClassification,AutoProcessor
from transformers.data.data_collator import default_data_collator
from typing import Union
# Project specific objects
from paper_solver.preprocess import *

In [4]:
#| export
from datasets import load_from_disk
train_dataset = load_from_disk(OUTPUT_PATH/'train_split')
eval_dataset = load_from_disk(OUTPUT_PATH/'eval_split')

In [5]:
#| export
## Loading raw dataset without Encoding or applying feature extractor
train_raw_dataset = load_from_disk(OUTPUT_PATH/'raw_data/train')
eval_raw_dataset = load_from_disk(OUTPUT_PATH/'raw_data/test')

In [6]:
train_raw_dataset

Dataset({
    features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
    num_rows: 8
})

In [8]:
train_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 8
})

## Modelling

### Adding labels and loading Model

In [9]:
#| export
label_list = train_dataset.features["labels"].feature.names
num_labels = len(label_list)
label2id, id2label = dict(), dict()
for i, label in enumerate(label_list):
    label2id[label] = i
    id2label[i] = label

In [10]:
#| export
from transformers import LiltForTokenClassification
model_id = "SCUT-DLVCLab/lilt-roberta-en-base"
# load model with correct number of labels and mapping
model = LiltForTokenClassification.from_pretrained(
    model_id, num_labels=len(label_list), label2id=label2id, id2label=id2label
)


Some weights of LiltForTokenClassification were not initialized from the model checkpoint at SCUT-DLVCLab/lilt-roberta-en-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Defining Metric

In [13]:
#| export
metric = load_metric("seqeval")
return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(
        predictions=true_predictions, 
        references=true_labels,
        zero_division='0'
    )
    if return_entity_level_metrics:
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

/tmp/ipykernel_153425/4115166580.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


### Training

In [16]:
#| export
from transformers import Trainer, TrainingArguments
NUM_TRAIN_EPOCHS = 100
PER_DEVICE_TRAIN_BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH_SIZE = 4
LEARNING_RATE = 4e-5
training_args = TrainingArguments(output_dir="LiLT_INVOICE",
                                  # max_steps=1500,
                                  num_train_epochs=NUM_TRAIN_EPOCHS,
                                  logging_strategy="epoch",
                                  save_total_limit=1,
                                  per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                                  per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                                  learning_rate=LEARNING_RATE,
                                  evaluation_strategy="no",
                                  save_strategy="no",
                                  # eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)


In [17]:
#| export
assert torch.cuda.is_available()
import gc
torch.cuda.empty_cache()
gc.collect()

494

Start training

In [18]:
#| export
trainer.train()

The following columns in the training set don't have a corresponding argument in `LiltForTokenClassification.forward` and have been ignored: pixel_values. If pixel_values are not expected by `LiltForTokenClassification.forward`,  you can safely ignore this message.
/home/prabhanshu/mambaforge/envs/fastai/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 200
  Number of trainable parameters = 130179027


Step,Training Loss
2,3.018400
4,2.335000
6,1.628000
8,1.366000
10,1.188600
12,1.020800
14,0.814500
16,0.651600
18,0.521600
20,0.462600




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.1665590265695937, metrics={'train_runtime': 76.1284, 'train_samples_per_second': 10.509, 'train_steps_per_second': 2.627, 'total_flos': 222552910233600.0, 'train_loss': 0.1665590265695937, 'epoch': 100.0})

## Evaluation

In [19]:
#| export
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `LiltForTokenClassification.forward` and have been ignored: pixel_values. If pixel_values are not expected by `LiltForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 4


{'eval_loss': 3.049816131591797,
 'eval_precision': 0.1746031746031746,
 'eval_recall': 0.5,
 'eval_f1': 0.2588235294117647,
 'eval_accuracy': 0.4883116883116883,
 'eval_runtime': 0.1528,
 'eval_samples_per_second': 26.177,
 'eval_steps_per_second': 6.544,
 'epoch': 100.0}

## Saving Trained Model

In [21]:
#| export
trainer.save_model(OUTPUT_PATH/'LiLTmodel')

Saving model checkpoint to ../data/preprocessed/LiLTmodel
Configuration saved in ../data/preprocessed/LiLTmodel/config.json
Model weights saved in ../data/preprocessed/LiLTmodel/pytorch_model.bin


In [22]:
#| hide
import nbdev; nbdev.nbdev_export()